# Notizbuch für den 'data' Endpunkt & Spezifikation von 'job=true'
Initial ist 'job=false' gesetzt, daher muss dieser Parameter bei der Abfrage eins zu großen Datensatzes angepasst werden.

In [4]:
import json
import logging
from pygenesis.http_helper import load_data
from pygenesis.table import Table

In [5]:
# set logging level to INFO to receive more detailed program response
logging.basicConfig(level=logging.INFO)

endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0001',
          'area':'all',
          'timeslices':20,
          'job':'false',
          'language':'de',}
response = load_data(endpoint=endpoint, method=method, params=params, as_json=True)

In [6]:
print(response)

{'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 99, 'Content': 'Der Bearbeitungsauftrag wurde erstellt. Die Tabelle kann in Kürze als Ergebnis mit folgendem Namen abgerufen werden: 42151-0001_609089248', 'Type': 'Information'}, 'Parameter': {'username': '********************', 'password': '********************', 'name': '42151-0001', 'area': 'Alle', 'compress': 'false', 'transpose': 'false', 'startyear': '', 'endyear': '', 'timeslices': '20', 'regionalvariable': '', 'regionalkey': '', 'classifyingvariable1': '', 'classifyingkey1': '', 'classifyingvariable2': '', 'classifyingkey2': '', 'classifyingvariable3': '', 'classifyingkey3': '', 'format': 'csv', 'job': 'true', 'stand': '01.01.1970 01:00', 'language': 'de'}, 'Object': None, 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}


## Hier endet die Funktionalität des Pakets, der*die User muss wie folgt fortfahren

Der User überprüft den Status des Jobs mittels des catalogue/jobs.

In [1]:
job_id = response.get("Status").get("Content").split(":")[1].strip()

NameError: name 'response' is not defined

In [5]:
endpoint = "catalogue"
method = "jobs"
catalogue_params = {"selection":f"*{job_id}",
          "sortcriterion":"Auftragstyp",
          "language":"de",}
catalogue_response = load_data(endpoint=endpoint, method=method, params=catalogue_params, as_json=True)

INFO:pygenesis.http_helper:Code 0: erfolgreich


In [6]:
print(catalogue_response)

{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 0, 'Content': 'erfolgreich', 'Type': 'Information'}, 'Parameter': {'username': '********************', 'password': '********************', 'selection': '*42151-0001_609089248', 'searchcriterion': 'Code', 'sortcriterion': 'Auftragstyp', 'type': 'Alle', 'pagelength': '100', 'language': 'de', 'area': 'Benutzer'}, 'List': [{'Content': 'Werteabruf', 'Date': '01.10.2022', 'Time': '09:24:00', 'State': 'Fertig', 'Code': '42151-0001_609089248'}], 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}


Ist der Status 'Fertig' so kann abschließend das resultfile über den data/resultfile Endpunkt abgerufen werden.

In [7]:
endpoint = "data"
method = "resultfile"
resultfile_params = {"name": job_id,
                     "area":"all",
                     "language":"de",}
result = load_data(endpoint=endpoint, method=method, params=resultfile_params)

Dieses Resultat wird gecached.

In [8]:
print(result)

GENESIS-Tabelle: 42151-0001
Auftragseingang im Verarbeitenden Gewerbe (Wertindex):;;;;;;;;;;;;;;;;;;;;;;;;;;
Deutschland, Monate, Original- und bereinigte Daten,;;;;;;;;;;;;;;;;;;;;;;;;;;
Absatzrichtung, Wirtschaftszweige (Hauptgr. und Aggregate);;;;;;;;;;;;;;;;;;;;;;;;;;
Indizes des Auftragseingangs im Verarb. Gewerbe;;;;;;;;;;;;;;;;;;;;;;;;;;
Deutschland;;;;;;;;;;;;;;;;;;;;;;;;;;
Auftragseingang: Wertindex (2015=100);;;;;;;;;;;;;;;;;;;;;;;;;;
;;Original- und bereinigte Daten;;;;;;;;;;;;;;;;;;;;;;;;
;;Originalwerte;;;;;X13 JDemetra+  kalenderbereinigt;;;;;X13 JDemetra+  kalender- und saisonbereinigt;;;;;BV4.1 kalender- und saisonbereinigt;;;;;BV4.1 Trend;;;;
;;Absatzrichtung;;;;;Absatzrichtung;;;;;Absatzrichtung;;;;;Absatzrichtung;;;;;Absatzrichtung;;;;
;;Insgesamt;Inland;Ausland;Ausland (Eurozone);Ausland (Nicht-Eurozone);Insgesamt;Inland;Ausland;Ausland (Eurozone);Ausland (Nicht-Eurozone);Insgesamt;Inland;Ausland;Ausland (Eurozone);Ausland (Nicht-Eurozone);Insgesamt;Inland;Ausland;A

Da wir initial eine Tabelle angefragt hatten, können wir diese in einen DataFrame mittels der entsprechenden Klasse verwandeln.

In [9]:
r = Table(job_id)

In [10]:
r.get_data()

In [11]:
r.metadata

{'Ident': {'Service': 'metadata', 'Method': 'table'},
 'Status': {'Code': 22,
  'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: name)',
  'Type': 'Warnung'},
 'Parameter': {'username': '********************',
  'password': '********************',
  'name': '42151-0001',
  'area': 'Alle',
  'language': 'de'},
 'Object': {'Code': '42151-0001',
  'Content': 'Auftragseingang im Verarbeitenden Gewerbe (Wertindex):\nDeutschland, Monate, Original- und bereinigte Daten,\nAbsatzrichtung, Wirtschaftszweige (Hauptgr. und Aggregate)',
  'Time': {'From': '1991', 'To': '2022'},
  'Valid': 'false',
  'Structure': {'Head': {'Code': '42151',
    'Content': 'Indizes des Auftragseingangs im Verarb. Gewerbe',
    'Type': 'Statistik',
    'Values': None,
    'Selected': None,
    'Structure': [{'Code': 'DINSG',
      'Content': 'Deutschland insgesamt',
      'Type': 'Merkmal',
      'Values': '1',
      'Selected': '1',
     

In [12]:
r.data

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,...,3_Auspraegung_Label,4_Merkmal_Code,4_Merkmal_Label,4_Auspraegung_Code,4_Auspraegung_Label,5_Merkmal_Code,5_Merkmal_Label,5_Auspraegung_Code,5_Auspraegung_Label,AUF101__Auftragseingang:_Wertindex__2015=100
0,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2021,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Insgesamt,WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",WZ08-C,Verarbeitendes Gewerbe,MONAT,Monate,MONAT01,Januar,"104,6"
1,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2021,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Insgesamt,WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",WZ08-C,Verarbeitendes Gewerbe,MONAT,Monate,MONAT02,Februar,"111,6"
2,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2021,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Insgesamt,WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",WZ08-C,Verarbeitendes Gewerbe,MONAT,Monate,MONAT03,März,"134,8"
3,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2021,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Insgesamt,WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",WZ08-C,Verarbeitendes Gewerbe,MONAT,Monate,MONAT04,April,"118,0"
4,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2021,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Insgesamt,WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",WZ08-C,Verarbeitendes Gewerbe,MONAT,Monate,MONAT05,Mai,"111,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12595,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Ausland (Nicht-Eurozone),WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",GUT-INVEST-07,Investitionsgüter ohne 29 und 30,MONAT,Monate,MONAT08,August,...
12596,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Ausland (Nicht-Eurozone),WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",GUT-INVEST-07,Investitionsgüter ohne 29 und 30,MONAT,Monate,MONAT09,September,...
12597,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Ausland (Nicht-Eurozone),WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",GUT-INVEST-07,Investitionsgüter ohne 29 und 30,MONAT,Monate,MONAT10,Oktober,...
12598,42151,Indizes des Auftragseingangs im Verarb. Gewerbe,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,WERT03,...,Ausland (Nicht-Eurozone),WZ08Y1,"WZ2008 (Hauptgruppen, Aggregate): Verarb. Gewerbe",GUT-INVEST-07,Investitionsgüter ohne 29 und 30,MONAT,Monate,MONAT11,November,...
